# Phase 3: RAG with History-Aware Retrieval

## Objectives:
1. Collect and chunk Indian legal documents
2. Generate embeddings using sentence-transformers
3. Build FAISS vector index
4. Implement retrieval pipeline with LangChain
5. Test grounded responses with citations


In [ ]:
# Install dependencies
%pip install sentence-transformers faiss-cpu langchain langchain-community chromadb

# Import libraries
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import json
import pickle


## Step 1: Load Legal Documents


In [ ]:
# Load cleaned legal Q&A as base knowledge documents
def load_legal_documents():
    """Load legal documents for RAG"""
    documents = []
    
    # Load the cleaned dataset
    with open('../data/processed/lawbot_cleaned.jsonl', 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            # Create document with both question and answer as context
            doc_text = f"Question: {item['instruction']}\nAnswer: {item['output']}\nSource: {item['source']}"
            documents.append({
                'text': doc_text,
                'source': item['source'],
                'metadata': {'instruction': item['instruction'], 'output': item['output']}
            })
    
    print(f"Loaded {len(documents)} legal documents")
    return documents

documents = load_legal_documents()
print(f"Sample document: {documents[0]['text'][:200]}")


## Step 2: Chunk Documents


In [ ]:
# Chunk documents into smaller pieces (600-1000 tokens)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Approximate tokens
    chunk_overlap=100,  # Overlap for context preservation
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = []
metadata_list = []

for doc in documents:
    doc_chunks = text_splitter.split_text(doc['text'])
    for chunk in doc_chunks:
        chunks.append(chunk)
        metadata_list.append({
            'source': doc['source'],
            'instruction': doc['metadata']['instruction'],
            'output': doc['metadata']['output']
        })

print(f"Total chunks created: {len(chunks)}")
print(f"Average chunk length: {np.mean([len(c) for c in chunks]):.0f} characters")
print(f"\nSample chunk:\n{chunks[0][:300]}")


## Step 3: Generate Embeddings


In [ ]:
# Load sentence transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"Embedding model: all-MiniLM-L6-v2")
print(f"Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}")

# Generate embeddings for all chunks
print("Generating embeddings...")
embeddings = embedding_model.encode(chunks, show_progress_bar=True, batch_size=32)

print(f"Embeddings shape: {embeddings.shape}")
print(f"Embedding dimension: {embeddings.shape[1]}")


## Step 4: Build FAISS Index


In [ ]:
# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity search

# Convert to float32 for FAISS
embeddings_f32 = embeddings.astype('float32')

# Add embeddings to index
index.add(embeddings_f32)

print(f"FAISS index created with {index.ntotal} vectors")
print(f"Index dimension: {dimension}")


## Step 5: Save FAISS Index


In [ ]:
# Save FAISS index
import os
os.makedirs('../vectorstore/faiss_index', exist_ok=True)

faiss.write_index(index, '../vectorstore/faiss_index/faiss_index.idx')

# Save chunks and metadata
with open('../vectorstore/faiss_index/chunks.pkl', 'wb') as f:
    pickle.dump(chunks, f)

with open('../vectorstore/faiss_index/metadata.pkl', 'wb') as f:
    pickle.dump(metadata_list, f)

# Save embedding model path
with open('../vectorstore/faiss_index/config.json', 'w') as f:
    json.dump({
        'model_name': 'all-MiniLM-L6-v2',
        'dimension': dimension,
        'num_vectors': len(chunks),
        'chunk_size': 800,
        'chunk_overlap': 100
    }, f, indent=2)

print("FAISS index and metadata saved successfully!")


## Step 6: Implement Retrieval Pipeline


In [ ]:
def retrieve_relevant_chunks(query, index, embedding_model, chunks, metadata_list, k=5):
    """Retrieve top-k relevant chunks for a query"""
    # Encode query
    query_embedding = embedding_model.encode([query])
    query_embedding = query_embedding.astype('float32')
    
    # Search in FAISS
    distances, indices = index.search(query_embedding, k)
    
    # Retrieve relevant chunks
    relevant_chunks = []
    for idx in indices[0]:
        relevant_chunks.append({
            'chunk': chunks[idx],
            'metadata': metadata_list[idx],
            'distance': float(distances[0][idx] if idx < len(distances[0]) else float('inf'))
        })
    
    return relevant_chunks

# Test retrieval
test_query = "What is the procedure for filing a criminal case?"
results = retrieve_relevant_chunks(test_query, index, embedding_model, chunks, metadata_list, k=5)

print(f"Query: {test_query}")
print(f"\nRetrieved {len(results)} relevant chunks:")
for i, result in enumerate(results, 1):
    print(f"\nChunk {i} (distance: {result['distance']:.4f}):")
    print(f"Source: {result['metadata']['source']}")
    print(f"Text: {result['chunk'][:200]}...")


## Step 7: Query Reformulation with History


In [ ]:
def reformulate_query(query, conversation_history):
    """Reformulate query based on conversation history"""
    if not conversation_history:
        return query
    
    # Simple reformulation: prepend context from conversation
    context = " ".join([f"{h['role']}: {h['content']}" for h in conversation_history[-3:]])  # Last 3 turns
    reformulated = f"{context} Current question: {query}"
    
    return reformulated

# Test query reformulation
conversation_history = [
    {"role": "user", "content": "What is IPC?"},
    {"role": "assistant", "content": "IPC stands for Indian Penal Code."},
    {"role": "user", "content": "What are the main offences?"}
]

original_query = "What are the punishments for theft?"
reformulated = reformulate_query(original_query, conversation_history)

print(f"Original query: {original_query}")
print(f"\nReformulated query: {reformulated}")


## Step 8: Generate Grounded Responses


In [ ]:
def generate_grounded_response(query, conversation_history, index, embedding_model, chunks, metadata_list, k=5):
    """Generate response with citations from retrieved chunks"""
    # Reformulate query
    reformulated_query = reformulate_query(query, conversation_history)
    
    # Retrieve relevant chunks
    retrieved = retrieve_relevant_chunks(reformulated_query, index, embedding_model, chunks, metadata_list, k)
    
    # Check if we have sufficient evidence
    if retrieved and retrieved[0]['distance'] < 2.0:  # Threshold for relevance
        # Build context from retrieved chunks
        context = "\n\n".join([f"[{i+1}] {r['chunk']}\nSource: {r['metadata']['source']}" 
                                for i, r in enumerate(retrieved)])
        
        # Format response with citations
        citations = [r['metadata']['source'] for r in retrieved]
        unique_citations = list(set(citations))
        
        response = {
            'answer': f"Based on the following legal sources: {', '.join(unique_citations)}",
            'context': context,
            'citations': unique_citations,
            'confidence': 'high' if retrieved[0]['distance'] < 1.0 else 'medium'
        }
    else:
        # Ab-stain response
        response = {
            'answer': "I don't have sufficient legal information to provide an accurate answer to this question.",
            'context': '',
            'citations': [],
            'confidence': 'low'
        }
    
    return response

# Test grounded response generation
test_queries = [
    "What is the punishment for murder under IPC?",
    "How do I file a criminal complaint?",
    "What is the Constitution of India about?"
]

for query in test_queries:
    response = generate_grounded_response(query, [], index, embedding_model, chunks, metadata_list)
    print(f"\nQuery: {query}")
    print(f"Answer: {response['answer']}")
    print(f"Citations: {response['citations']}")
    print(f"Confidence: {response['confidence']}")
    if response['context']:
        print(f"Context preview: {response['context'][:200]}...")
    print("-" * 80)


## Summary

Phase 3 completed successfully! RAG pipeline has been:
1. ✅ Loaded and chunked legal documents
2. ✅ Generated embeddings using sentence-transformers
3. ✅ Built FAISS vector index
4. ✅ Implemented retrieval with top-k search
5. ✅ Added query reformulation with conversation history
6. ✅ Generated grounded responses with citations
7. ✅ Implemented confidence threshold for abstention

**Deliverables:**
- `vectorstore/faiss_index/faiss_index.idx` - FAISS vector index
- `vectorstore/faiss_index/chunks.pkl` - Document chunks
- `vectorstore/faiss_index/metadata.pkl` - Metadata
- `vectorstore/faiss_index/config.json` - Configuration
- RAG retrieval pipeline ready for integration
